In [1]:
from ore_algebra import *

from ore_algebra.analytic.differential_operator import DifferentialOperator
from ore_algebra.analytic.local_solutions import (log_series, LocalBasisMapper,
                                                  simplify_exponent, LogMonomial)
from ore_algebra.analytic.path import Point

In [2]:
import itertools as it

def group_by_module(roots):
    roots.sort(key=abs)
    return it.groupby(roots, key=abs)

In [3]:
def my_expansions(op, point, order=None, ring=None):
    mypoint = Point(point, op)
    dop = DifferentialOperator(op)
    ldop = dop.shift(mypoint)
    if order is None:
        ind = ldop.indicial_polynomial(ldop.base_ring().gen())
        order = max(dop.order(), ind.dispersion()) + 3
    class Mapper(LocalBasisMapper):
        def fun(self, ini):
            return log_series(ini, self.shifted_bwrec, order)
    sols = Mapper(ldop).run()
    x = SR.var(dop.base_ring().variable_name())
    dx = x if point.is_zero() else x.add(-point, hold=True)
    if ring is None:
        cm = get_coercion_model()
        ring = cm.common_parent(
                dop.base_ring().base_ring(),
                mypoint.value.parent(),
                *(sol.leftmost for sol in sols))
    res = [[(c / ZZ(k).factorial() * (-point)^sol.leftmost, point, sol.leftmost, k)
                    for n, vec in enumerate(sol.value)
                    for k, c in reversed(list(enumerate(vec)))]
           for sol in sols]
    return res

##### Prop 2.17 (p66) :
Si
$F(z) \sim C {(1 - \frac{z}{\zeta})}^\alpha \log^m (1 - \frac{z}{\zeta})$ avec $\alpha \not\in \mathbb{N}$, alors
$f_n \sim C \zeta^{-n} \frac{n^{-\alpha - 1}}{\Gamma(-\alpha)} \log^m(n)$.

In [4]:
class GeneralMonomial:
    def __init__(self, c, zeta, alpha, m):
        self.c = c
        self.zeta = zeta
        self.alpha = alpha
        self.m = m
        # TODO assert alpha not in NN ou gerer différemment
        self.asymp_c = c / gamma(-alpha)

    def is_singular(self):
        return self.alpha < 0 or self.m != 0

    def __repr__(self):
        if self.m == 0:
            return f'{self.asymp_c} * {1/self.zeta}^n * n^{-self.alpha - 1}'
        return f'{self.asymp_c} * {1/self.zeta}^n * n^{-self.alpha - 1} * log^{self.m}(n)'

    def asymptotic_symbolic(self):
        n = var('n')
        return self.c / gamma(-self.alpha) * self.zeta^n * n^(-self.alpha - 1) * log(n)^self.m

    def __cmp__(self, other):
        # TODO zeta et alpha algébriques, c'est bon comme méthode de comparaison ?
        biggest = None
        if self.zeta != other.zeta:
            biggest = self if self.zeta < other.zeta else other  # le plus petit zeta gagne
        elif self.alpha != other.alpha:
            biggest = self if self.alpha < other.alpha else other  # le plus petit alpha gagne
        elif self.m != other.m:
            biggest = self if self.m > other.m else other  # le plus grand m gagne
        if biggest is None or not biggest.c.is_nonzero():
            return 0
        return 1 if biggest is self else -1
    
    def __lt__(self, other):
        return 1 if self.__cmp__(other) == -1 else 0

In [5]:
# Create the differential algebra to encode linear differential equation
Pols.<z> = PolynomialRing(QQ)
Diff.<Dz> = OreAlgebra(Pols)

diffWalk = (4*z^2 - z)*Dz^2 + (14*z - 2)*Dz + 6
print(diffWalk)
basis_in_0 = diffWalk.local_basis_expansions(0)
print(basis_in_0)
ini = [0, 1]  # TODO

(4*z^2 - z)*Dz^2 + (14*z - 2)*Dz + 6
[z^(-1), 1 + 3*z + 10*z^2 + 35*z^3]


In [6]:
roots = diffWalk.leading_coefficient().roots(QQbar, multiplicities=False)
roots.sort(key=abs)

eps = 1e-20
found_a_sing = False
all_monomials = []  # defini en dehors pour tenir compte de tous les monômes potentiels
for module, roots in group_by_module(roots):
    roots = list(roots)
    print(f'module: {module}, roots: {roots}')
    if module == 0:
        continue
    for root in roots:
        trans_matrix = diffWalk.numerical_transition_matrix([0, root], eps=eps)
        coeffs_in_local_basis = trans_matrix * vector(ini)
        
        local_expansions = my_expansions(diffWalk, root)
        for k, terms in enumerate(local_expansions):
            local_expansions[k] = terms[0]  # pousser plus loin ?
        
        all_monomials += [GeneralMonomial(lambda_i * c, zeta, alpha, m)
                         for lambda_i, (c, zeta, alpha, m) in zip(coeffs_in_local_basis, local_expansions)]
        
        if any(coeff.is_nonzero() for coeff in coeffs_in_local_basis):
            found_a_sing = True

    if found_a_sing:
        all_monomials.sort(reverse=True)
        print(f'all monomials: {all_monomials}')
        break

module: 0, roots: [0]
module: 0.25000000000000000?, roots: [0.25000000000000000?]
all monomials: [([2.00000000000000000000 +/- 1.70e-21])/sqrt(pi) * 4^n * n^-1/2, 0 * 4^n * n^-1]
